In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D
from keras.models import Model
from keras import backend as K
from sklearn.utils import resample

n_features = 4
latent_dim = 10
n_runs = 300
max_len = 0

def prepare_training(path, n_runs):
    labels = []
    def closest_4(n, m):
        q = n / m
        n1 = m * q
        if (n * m) > 0:
            n2 = m * (q + 1)
        else:
            n2 = m * (q - 1)
        if abs(n-n1) < abs(n-n2):
            return int(n1)
        return int(n2)
    
    
    def extend_line(run, max_len):
        difference = abs(len(run) - max_len)
        extension = np.array([run[-1]]*difference)
        if difference != 0:
            run = np.vstack([run, extension])
        return run
    
    def get_max_len(sequence_list):
        max_len = 0
        min_len = 1000
        for seq in sequence_list:
            if len(seq) > max_len:
                max_len = len(seq)
            if len(seq) < min_len:
                min_len = len(seq)
        return max_len, min_len
    
    def construct_matrix(sequence_list):
        max_len, min_len = get_max_len(sequence_list)
        print(max_len)
        len = closest_4(max_len,4)
        len = 440
        train_matrix = np.zeros(shape=(n_runs, len, n_features))
        for index, run in enumerate(sequence_list):
            line = extend_line(run, len)
            train_matrix[index] = line
        return train_matrix
        
        
    def stadard_sequences(seqs):
        for i, seq in enumerate(seqs):
            #seqs[i] = MinMaxScaler(feature_range=[0, 1]).fit_transform(seq)
            seqs[i] = StandardScaler().fit_transform(seq)
        return seqs       
    
    
    def order_runs_by_len(runs):
        runs.sort(key=len)
        for r in runs:
            labels.append(r['Choice'][0])
        for i,r in enumerate(runs):
            runs[i] = runs[i].drop(columns=['Choice'])
        return runs, labels
    
    def read_sequences():
        run_list_mix = []
        for index in range(n_runs):
            run_csv = pd.read_csv(path+str(index))
            run_csv = run_csv.drop(columns=['Unnamed: 0'])
            run_list_mix.append(run_csv)
        run_list_ordered, labels = order_runs_by_len(run_list_mix)
        stands = stadard_sequences(run_list_ordered)
        padded_matrix = construct_matrix(stands)
        return padded_matrix, labels
    
    return read_sequences()

train_matrix, labels = prepare_training("Mix_sequences_var_length/run^", n_runs=n_runs) 
labels = np.array(labels)



Using TensorFlow backend.


412


In [26]:
import keras
from keras import objectives
from keras.layers import Input, LSTM, RepeatVector, Conv2DTranspose, MaxPooling1D, UpSampling1D, AveragePooling1D
from keras.losses import mse
from keras.models import Model


filters = 50
intermediate_dimension = 50 
latent_dim = 10


def Conv1DTranspose(input_tensor, filters, kernel_size,strides=1, padding='same'):
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1),
                            activation='relu',
                            strides=strides, padding='same')(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x


def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim), mean=0., stddev=0.00001)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon
# 
# 
# def repeat(x):
#     steps_matrix = K.ones_like(x[0][:, :, :1])
#     
#     latent_matrix = K.expand_dims(x[1], axis=1)
#     return K.batch_dot(steps_matrix, latent_matrix)


def create_vae():
    inputs = Input(shape=(train_matrix.shape[1], n_features), name='Vae_input')
    x = inputs
    
    for i in range(2):
        x = Conv1D(filters=filters,
                   kernel_size=20,
                   activation='relu',
                   padding='same')(x)
        x = MaxPooling1D(pool_size=2)(x)
    shape = K.int_shape(x)
    # encoder_outputs, state_h, state_c = LSTM(latent_dim, return_state=True)(x)
    # encoder_states = [state_h, state_c]     
    x, st_1,st_2 = LSTM(intermediate_dimension, return_state=True)(x)
    states = [st_1, st_2]
    emb = Dense(latent_dim)(x)
    # z_mean = Dense(latent_dim, name='z_mean',)(state_h)
    # z_log_var = Dense(latent_dim, name='z_log_var')(state_h)
    # z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
    
    encoder = Model(inputs, emb, name='encoder')
    
    # 
    #latent_inputs = Input(shape=(latent_dim,), name='latent_inputs')
    # x_dec = Dense(shape[1]*shape[2])(z)
    # x_dec = Reshape((shape[1], shape[2]))(x_dec)
    
    # lat_inputs = Input((latent_dim,), name='dec_inputs')
    # x_dec = Dense(shape[1]*shape[2], name='dense_dec')(lat_inputs)
    # x_dec = Reshape((shape[1], shape[2]))(x_dec)
    # #decoder_inputs = Dense(latent_dim, name='Dense_latent_pre_LSTM')(decoder_inputs)
    
    #latent_in = Input((latent_dim,), name='latent_input')
    x = Dense(shape[1]*shape[2])(emb)
    x = Reshape((shape[1], shape[2]))(x)
    dec_lstm = LSTM(shape[2], return_sequences=True)
    x = dec_lstm(x, initial_state=states)
    x_dec = x
    # decoded = RepeatVector(train_matrix.shape[1])(encoder_outputs)
    # decoded_lstm = LSTM(latent_dim, return_sequences=True)
    # x_dec = decoded_lstm(decoded, initial_state=encoder_states)
    
    
    # decoder_inputs = Input((train_matrix.shape[1], n_features), name='dec_input')
    # dec_lstm = LSTM(latent_dim, return_sequences=True)
    # x_dec = dec_lstm(decoder_inputs, initial_state=encoder_states)
    #dec_lstm = decoder_lstm(decoder_inputs)
    #print(K.int_shape(dec_lstm))
    
    # x_dec = decoder_lstm
    for i in range(2):
        x_dec = Conv1DTranspose(input_tensor=x_dec,
                            filters=filters,
                            kernel_size=20,
                            padding='same')
        x_dec = UpSampling1D(size=2)(x_dec)
        
    output = Dense(n_features, name='model_output')(x_dec)
    # decoder = Model(latent_in, output)
    # output = decoder(encoder.output)

    # def vae_loss(x, x_decoded_mean):
    #     xent_loss = objectives.mean_squared_error(x, x_decoded_mean) 
    #     #xent_loss = K.sum(K.binary_crossentropy(x_decoded_mean, x), axis=-1)
    #     kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var))
    #     loss = xent_loss + 0*kl_loss
    #     return loss
    
    vae = Model(inputs, output, name='vae')
   
    vae.compile(optimizer='adam', loss='mse')
    
    vae.summary()
    
    #vae.save_weights("Models/Weights/AE_CONV_LSTM_Diff_len_dist_MATRIX_LEN.hdf5")
    return vae, encoder

model, encoder = create_vae()
   
print("Done")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Vae_input (InputLayer)          (None, 440, 4)       0                                            
__________________________________________________________________________________________________
conv1d_19 (Conv1D)              (None, 440, 50)      4050        Vae_input[0][0]                  
__________________________________________________________________________________________________
max_pooling1d_19 (MaxPooling1D) (None, 220, 50)      0           conv1d_19[0][0]                  
__________________________________________________________________________________________________
conv1d_20 (Conv1D)              (None, 220, 50)      50050       max_pooling1d_19[0][0]           
__________________________________________________________________________________________________
max_poolin

In [27]:
def return_mask(num, labs):
    arg = np.squeeze(np.argwhere(labs == num))
    return arg

masks = [return_mask(num, np.array(labels)) for num in range(0, 9)]


def plot_enc(encoder):
  
    
    
    from sklearn.decomposition import PCA
    
    
    
    encodings = encoder.predict(train_matrix)
    
    enc_mean, enc_var, z_enc = encodings, encodings, encodings
    #enc_mean, enc_var, z_enc = encodings[0], encodings[1], encodings[0]
    
    print(enc_mean.shape, enc_var.shape, z_enc.shape)
    
    
    from mpl_toolkits.mplot3d import Axes3D  
    
    
    
    def plot_pca(title, i): 
        # 
        # fig = plt.figure()
        # ax = fig.add_subplot(111, projection='3d')
        # markers = ['o', 'o', 'o', 'o', '^', '^', '^', '^', '^', '^']
        # for index, mask in enumerate(masks):
        #     
        #     ax.scatter(principalComponents[:, 0][mask], 
        #                principalComponents[:, 1][mask],  
        #                principalComponents[:, 2][mask], marker=markers[index])
        # # for mask in unseen_mask:
        # #     ax.scatter(unseen_encoding[0][:,0][mask],
        # #                unseen_encoding[0][:,1][mask],
        # #                unseen_encoding[0][:,2][mask])
        # plt.legend(labels=np.arange(0, 9))
        # plt.title(str(title))
        # plt.show()
        
        for mask in masks:
            plt.scatter(x=principalComponents[:, 0][mask], 
                        y=principalComponents[:, 1][mask],
                        alpha=0.5)
        # for mask in unseen_mask:
        #     plt.scatter(unseen_encoding[0][:,0][mask],
        #            unseen_encoding[0][:,1][mask])
        #     
            #break
        
        plt.legend(labels=np.arange(0, 9))
        plt.title(str(title))
        plt.show()
    
    
    enc_list = [enc_mean, enc_var, z_enc]
    titles = ["MEAN","LOG_VAR","SAMPLED"]
    for i,enc in enumerate(enc_list):
        scaler = StandardScaler()
        enc_input = scaler.fit_transform(enc) 
        pca = PCA(3)
        principalComponents = pca.fit_transform(enc_input)
        print(principalComponents.shape)
        print(pca.explained_variance_ratio_)
        plot_pca('Sequences'+titles[i], 0)
        
        # principalComponents = enc
        # plot_pca('Sequences_Not_Pca'+titles[i], 0)
    


In [30]:


model.fit(train_matrix, train_matrix, epochs=10, verbose=1)
plot_enc(encoder)
        


# def train(model, ep):
#     print(train_matrix.shape)
#     model.fit(train_matrix,train_matrix, epochs=ep, verbose=1)
#     model.save_weights("Models/Weights/AE_CONV_LSTM_Diff_len_dist_MATRIX_LEN.hdf5")
# 
# for ep in enumerate(epochs):
#     train(model[0], i)

Epoch 1/10


 32/300 [==>...........................] - ETA: 28s - loss: 0.1810

 64/300 [=====>........................] - ETA: 23s - loss: 0.1633

 96/300 [========>.....................] - ETA: 17s - loss: 0.1759

128/300 [===========>..................] - ETA: 15s - loss: 0.1690

...

288/300 [===========================>..] - ETA: 2s - loss: 0.1342

300/300 [==============================] - 50s 167ms/step - loss: 0.1359


(300, 10) (300, 10) (300, 10)
(300, 3)
[0.76638836 0.18889004 0.03385089]


(300, 3)
[0.76638836 0.18889004 0.03385089]


(300, 3)
[0.76638836 0.18889004 0.03385089]


In [13]:

unseen_sequences_matrix, unseen_labs = prepare_training("Mix_sequences_var_length/run_unseen^", 
                                           n_runs=300)
unseen_mask = [return_mask(num, np.array(unseen_labs)) for num in range(0, 9)]

unseen_encoding = encoder.predict(unseen_sequences_matrix)

412


In [14]:
reconstruction = model.predict(train_matrix)
print(reconstruction.shape)
#RECONSTRUCTION
def reconstruct_sequence(seq_index):
    run = train_matrix[seq_index]
    #mask_seq = np.squeeze(np.argwhere(np.mean(run, axis=1) != 0))
    rec_run = reconstruction[seq_index]#[mask_seq]
    #print(rec_run[:,0])
    df = pd.DataFrame(rec_run[:-15], columns=["Sin", "Cosin", "Lat", "Lon"])
    df_original = pd.DataFrame(run[:-15], columns=["Sin", "Cosin", "Lat", "Lon"])
    plt.plot(df_original['Lon'], df_original['Lat'])
    plt.plot(df['Lon'], df['Lat'])
    plt.show()

for i in range(len(train_matrix)):
    reconstruct_sequence(i)
    if i == 5:
        break


(300, 440, 4)


In [78]:
def decode(encoded):
    encoded = np.reshape(encoded, (1,encoded.shape[0]))
    encodeds = decoder.predict(encoded)
    return encodeds

enc = encoder.predict(train_matrix)
# #print(enc[0])
# [-1.5779812, -0.962587, 0.49669492, -2.2342153, 1.8368481, 0.7021597,
#          0.59447, -0.8751026, -2.00302, 0.17431471]
r = enc[0]
print(r)
# r = [0]*10
r = np.array(r)
run = decode(r)
print(run.shape)
run = np.reshape(run, (run.shape[1], n_features))
print(run.shape)
run_df = pd.DataFrame(run, columns=["Sin", "Cosin", "Lat", "Lon"])
plt.plot(run_df['Lon'], run_df['Lat'])
plt.plot(run_df['Lon'], run_df['Lat'])
plt.show()

[[ 1.5563742   1.8690135   1.2696669  ...  0.74212474 -1.1532805
   0.8448648 ]
 [ 1.5491477   1.8579319   1.247019   ...  0.7297301  -1.1502178
   0.8230395 ]
 [ 1.5543007   1.8626639   1.245459   ...  0.71910673 -1.1549585
   0.8173853 ]
 ...
 [ 1.5153011   1.8728168   1.2646902  ...  0.7779858  -1.1397209
   0.9302615 ]
 [ 1.5245515   1.86223     1.2149523  ...  0.7070699  -1.1524011
   0.8437313 ]
 [ 1.530245    1.8662534   1.2199845  ...  0.70683646 -1.154669
   0.84412104]]


ValueError: cannot reshape array of size 5000 into shape (1,500)

In [18]:
reconstruction = model.predict(train_matrix)
reconstruction_unseen = model.predict(unseen_sequences_matrix)

#RECONSTRUCTION ERROR
def get_reconstructed_matrix(input_matrix, reconstrut):
    return_matrix = np.zeros(shape=input_matrix.shape)
    for i,run in enumerate(input_matrix):
        mask_seq = np.squeeze(np.argwhere(np.mean(run, axis=1) != 0))
        return_matrix[i][mask_seq] = reconstrut[i][mask_seq]
    return reconstrut


train_error = abs(train_matrix - get_reconstructed_matrix(train_matrix, reconstruction))
unseen_runs_error = abs(unseen_sequences_matrix - get_reconstructed_matrix(train_matrix, reconstruction_unseen))

train_error_avg = np.mean(train_error, axis=2)
unseen_error_avg = np.mean(unseen_runs_error, axis=2)
train_error_avg = np.mean(train_error_avg, axis=1)
unseen_error_avg = np.mean(unseen_error_avg, axis=1)
print(train_error_avg.shape, unseen_error_avg.shape)

for mask in masks:
    plt.scatter(np.linspace(1,n_runs,n_runs)[mask],train_error_avg[mask])
plt.title('ERROR ON TRAIN')
plt.show()

for mask in unseen_mask:
    plt.scatter(np.linspace(1,n_runs,n_runs)[mask],unseen_error_avg[mask])
plt.title('ERROR ON Unseen')
plt.show()


(300,) (300,)


In [20]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)

tsne_obj= tsne.fit_transform(enc_input)

print(tsne_obj.shape)

for mask in masks:
    plt.scatter(x=tsne_obj[:, 0][mask], 
                y=tsne_obj[:, 1][mask],
                alpha=0.5)
plt.show()


(300, 2)


In [31]:

reconstruction = model.predict([train_matrix, train_matrix])
reconstruction_unseen = model.predict([unseen_sequences_matrix, unseen_sequences_matrix])


#RECONSTRUCTION ERROR
def get_reconstructed_matrix(input_matrix, reconstrut):
    return_matrix = np.zeros(shape=input_matrix.shape)
    for i,run in enumerate(input_matrix):
        mask_seq = np.squeeze(np.argwhere(np.mean(run, axis=1) != 0))
        return_matrix[i][mask_seq] = reconstrut[i][mask_seq]    
    return reconstrut


train_error = abs(train_matrix-get_reconstructed_matrix(train_matrix,reconstruction))
unseen_runs_error = abs(unseen_sequences_matrix-get_reconstructed_matrix(train_matrix,reconstruction_unseen))

train_error_avg = np.mean(train_error, axis=2)
unseen_error_avg = np.mean(unseen_runs_error, axis=2)
train_error_avg = np.mean(train_error_avg, axis=1)
unseen_error_avg = np.mean(unseen_error_avg, axis=1)
print(train_error_avg.shape, unseen_error_avg.shape)

for mask in masks:
    plt.scatter(np.linspace(1,n_runs,n_runs)[mask],train_error_avg[mask])
plt.title('ERROR ON TRAIN')
plt.show()

for mask in unseen_mask:
    plt.scatter(np.linspace(1,n_runs,n_runs)[mask],unseen_error_avg[mask])
plt.title('ERROR ON Unseen')
plt.show()

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 2 arrays: [array([[[ 1.05906428, -0.3801914 , -1.6015229 ,  1.09292993],
        [ 1.06054695, -0.39865766, -1.55764168,  1.05214709],
        [ 1.06054695, -0.39865766, -1.51386392,  1.01591659],
        ...,
...

In [18]:
import ipyvolume as ipv
import numpy as np
x, y, z = unseen_encoding[:,0], unseen_encoding[:,1], unseen_encoding[:,2]

for mask in unseen_mask:
    ipv.scatter(x[mask], y[mask], z[mask], size=0.3, marker="sphere")
ipv.show()